In [1]:
import pandas as pd
import duckdb
import numpy as np

# Attention: count(*) et count(col) n'ont pas le même comportement

In [16]:
df = pd.DataFrame([[1,1,1,2,2,3,3],
                  [1,np.NaN,np.NaN,2,2,np.NaN,3]]
).T
df.columns = ["col1", "col2"]

In [17]:
df

,col1,col2
0,1.0,1.0
1,1.0,NaN
2,1.0,NaN
3,2.0,2.0
4,2.0,2.0
5,3.0,NaN
6,3.0,3.0


Count(*) comptabilise les Nulls !
exemple:

In [19]:
query = """
SELECT "col1", COUNT(*)
FROM df
GROUP BY "col1"
"""
duckdb.sql(query)

┌────────┬──────────────┐
│  col1  │ count_star() │
│ double │    int64     │
├────────┼──────────────┤
│    1.0 │            3 │
│    2.0 │            2 │
│    3.0 │            2 │
└────────┴──────────────┘

Si on réalise le COUNT() sur la colonne2, les résultats sont différents !

In [20]:
query = """
SELECT "col1", COUNT(col2)
FROM df
GROUP BY "col1"
"""
duckdb.sql(query)

┌────────┬─────────────┐
│  col1  │ count(col2) │
│ double │    int64    │
├────────┼─────────────┤
│    1.0 │           1 │
│    2.0 │           2 │
│    3.0 │           1 │
└────────┴─────────────┘

In [22]:
# Les deux valeurs nulles dans la colonne 2 pour la catégorie "1" ne sont pas prises en compte
df.head(3)

,col1,col2
0,1.0,1.0
1,1.0,NaN
2,1.0,NaN


## Exemple avec les appartements

In [24]:
appt_nord = pd.read_csv("data/appartements_nord_pdc.csv")

In [25]:
query = """
SELECT Commune, COUNT(*)
FROM appt_nord
GROUP BY Commune
"""
duckdb.sql(query).df().head()

,Commune,count_star()
0,MARCQ EN BAROEUL,335
1,SECLIN,43
2,LILLE,4513
3,VILLENEUVE-D'ASCQ,361
4,ERQUINGHEM-LYS,7


4513 ventes à Lille

In [28]:
# on insère volontairement un NULL 
appt_nord.loc[2, "valeur_fonciere"] = np.NaN
appt_nord.head(3)

,Date mutation,valeur_fonciere,No voie,Type de voie,Code voie,Voie,Code postal,Commune,Code departement,Code commune,Surface Carrez du 1er lot,Nombre pieces principales,Surface terrain
0,04/01/2022,208000.0,64.0,RUE,0010,DE L ABBE BONPAIN,59700.0,MARCQ EN BAROEUL,59,378,"57,69",3.0,NaN
1,04/01/2022,113000.0,1.0,RUE,0260,GUY MOCQUET,59113.0,SECLIN,59,560,"45,72",3.0,NaN
2,03/01/2022,NaN,22.0,RUE,8749,DE TREVISE,59000.0,LILLE,59,350,"62,83",3.0,NaN


In [30]:
# Le count n'a pas changé
query = """
SELECT Commune, COUNT(*)
FROM appt_nord
GROUP BY Commune
"""
duckdb.sql(query).df().head()

,Commune,count_star()
0,MARCQ EN BAROEUL,335
1,SECLIN,43
2,LILLE,4513
3,VILLENEUVE-D'ASCQ,361
4,ERQUINGHEM-LYS,7


En faisant le count sur une colonne particulière, les NULLS sont exclus du count ;)

In [32]:
query = """
SELECT Commune, COUNT(valeur_fonciere)
FROM appt_nord
GROUP BY Commune
"""
duckdb.sql(query).df().head()

,Commune,count(valeur_fonciere)
0,MARCQ EN BAROEUL,335
1,SECLIN,43
2,LILLE,4512
3,VILLENEUVE-D'ASCQ,361
4,ERQUINGHEM-LYS,7
